### Notes

Note, this requires using PyTorch v0.3.1.  Somewhere between 0.3.1 and 0.4.0 parts of the backend were significantly rewritten, preventing us from performing the following hacks. (Likely has to do with them fusing Variable and Tensor).  That may change once their new API stabilizes.

The nice thing about how this is working is that it should be general enough to work for compute, tree, and federated modes of Grid, depending on how the `receive` function works under the hood.

In [ ]:
from grid.clients.torch import TorchClient

/Users/jasonmancuso/anaconda/envs/openmined/lib/python3.6/site-packages/h5py-2.7.1-py3.6-macosx-10.7-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
client = TorchClient()


UPDATE: Connecting to IPFS... this can take a few seconds...

SUCCESS: Connected!!! - My ID: QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4

UPDATE: Querying known workers...
	WORKER: /p2p-circuit/ipfs/Qmaosc64H6Y29VFCFYJzJXCX9AuRp7RCsekLmajHNVEARD...SUCCESS!!!
	WORKER: /p2p-circuit/ipfs/QmQabt3SWuDvjse9z7GAcH2BGQv4wH8bumkd4x5oXN2obX...FAIL!!!

UPDATE: Searching for IPFS nodes - 27 found overall - 5 are OpenMined workers          

SUCCESS: Found 5 OpenMined nodes!!!

 ANCHOR - 0 - ID:VEARD  Ping:0.05sec  CPUs:1  CPU Load:4.2  Disk-util:97.7%  RAM-util:2.0%  GPUs:[]
COMPUTE - 1 - ID:LJqCp  Ping:0.40sec  CPUs:2  CPU Load:1.5  Disk-util:2.1%  RAM-util:86.4%  GPUs:[0 : Tesla K80 : 2/11439]
COMPUTE - 2 - ID:JGr7h  Ping:0.42sec  CPUs:2  CPU Load:1.5  Disk-util:2.1%  RAM-util:86.5%  GPUs:[0 : Tesla K80 : 2/11439]
COMPUTE - 3 - ID:K4Gbz  Ping:0.38sec  CPUs:2  CPU Load:2.9  Disk-util:2.1%  RAM-util:80.3%  GPUs:[0 : Tesla K80 : 10891/11439]
NODE    -   - - timeout - -


In [ ]:
service_self = client.services['torch_service']

In [ ]:
# service_self = client.services['torch_service']
# def hook_float_tensor___init__(service_self):
#     torch.FloatTensor.old___init__ = torch.FloatTensor.__init__
#     def new___init__(self, *args, **kwargs):
#         self.old___init__(*args, **kwargs)
#         self = service_self.register_object(self,False)

#     torch.FloatTensor.__init__ = new___init__

In [ ]:
import torch
import inspect
from torch.autograd import Variable
import random
from functools import wraps, partial, partialmethod
from types import *
import imp
# from contextlib import contextmanager

In [ ]:
def assign_workers_function(worker_ids):
    def decorate(func):
        @wraps(func)
        def send_to_workers(*args, **kwargs):
            part = func(*args, **kwargs)
            command = compile_command(part)
            for worker in worker_ids:
                print("Placeholder print for sending command to worker {}".format(worker))
                args, kwargs = send_command(command)
            receive_commands(worker_ids)  ## Probably needs to happen async
            return args, kwargs
        return send_to_workers
    return decorate

In [ ]:
def assign_workers_method(worker_ids):
    def decorate(method):
        @wraps(method)
        def send_to_workers(self, *args, **kwargs):
            part = method(self, *args, **kwargs)
            if self.is_pointer_to_remote:
                command = compile_command(part)
                for worker in worker_ids:
                    print("Placeholder print for sending command to worker {}".format(worker))
                    args, kwargs = send_command(command)
                receive_commands(worker_ids)  ## Probably needs to happen async
                return args, kwargs
            else:
                print('in there')
                print(service_self)
                result = part.func(*args, **kwargs)
                try:
                    getattr(result, 'id')
                    return result
                except:
                    return service_self.register_object(result, self.is_pointer_to_remote)
        return send_to_workers
    return decorate

In [ ]:
# # Slightly modified to remove parent class dependency
# torch.FloatTensor.old___init__ = torch.FloatTensor.__init__
# def hook_float_tensor___init__():
#     def new___init__(self, tensor, owner=client.services['torch_service'], *args, **kwargs):
#         super(torch.FloatTensor, self).__init__(*args, **kwargs)
#         self = owner.register_object(self, False)

#     torch.FloatTensor.__init__ = new___init__

In [ ]:
# service_self = client.services['torch_service']
# def hook_float_tensor___init__(service_self):
#     def new___init__(self, *args, **kwargs):
#         super(torch.FloatTensor, self).__init__(*args, **kwargs)
#         self = service_self.register_object(self,False)

#     torch.FloatTensor.__init__ = new___init__

In [ ]:
# def assign_workers_factory(worker_ids):
#     def decorate(method):
#         @wraps(method)
#         def send_to_workers(self, *args, **kwargs):
#             part = method(self, *args, **kwargs)
#             command = compile_command(part)
#             for worker in worker_ids:
#                 print("Placeholder print for sending command to worker {}".format(worker))
#                 args, kwargs = send_command(command)
#             receive_commands(worker_ids)  ## Probably needs to happen async
#             return old_init(*args, **kwargs)
#         return send_to_workers
#     return decorate

In [ ]:
def pass_func_args(func):
    @wraps(func)
    def pass_args(*args, **kwargs):
        return partial(func, *args, **kwargs)
    return pass_args

def pass_method_args(method):
    @wraps(method)
    def pass_args(*args, **kwargs):
        return partialmethod(method, *args, **kwargs)
    return pass_args

In [ ]:
def send_command(command):
    print(command)
    print('===========')
    print()
    return command['args'], command['kwargs']

def receive_commands(worker_ids):
    print('Placeholder print for receiving commands from workers in the following list')
    print(worker_ids)

In [ ]:
def compile_command(partial_func):
    func = partial_func.func
    args = partial_func.args
    kwargs = partial_func.keywords
    command = {}
    command['command'] = func.__name__
    command['command_type'] = type(func)
    command['args'] = args
    command['kwargs'] = kwargs
    command['arg_types'] = [type(x) for x in args]
    command['kwarg_types'] = [type(kwargs[x]) for x in kwargs]
    return command

In [ ]:
type(torch.FloatTensor)

In [ ]:
dir(object)

In [ ]:
torch.FloatTensor.__delattr__

In [ ]:
%%time

for attr in dir(torch):
    if attr == 'typename':
        continue
    if type(torch.__getattribute__(attr)) in [FunctionType, BuiltinFunctionType]:
        torch.__setattr__(attr, assign_workers_function(['A1','B1', 'B2'])(pass_func_args(torch.__getattribute__(attr))))
        
for attr in dir(torch.FloatTensor):
    lit = getattr(torch.FloatTensor, attr)
    is_desc = inspect.ismethoddescriptor(lit)
    is_func = type(lit)==FunctionType
    is_mappingproxy = attr == '__dict__'
    try:
        is_service_func = 'TorchService' in lit.__qualname__
    except:
        is_service_func = False
    is_base = attr in dir(object)
    is_none = lit is None
    if is_desc or (is_func and not is_service_func) and not is_base:
        print(attr)
        setattr(torch.FloatTensor, 'old_{}'.format(attr), lit)
        setattr(torch.FloatTensor, attr, assign_workers_method(['A1','B1', 'B2'])(pass_method_args(lit)))
    else:
        print(attr, ' skipped')

In [16]:
imp.reload(torch)

<module 'torch' from '/Users/jasonmancuso/anaconda/envs/openmined/lib/python3.6/site-packages/torch/__init__.py'>

In [28]:
for attr in dir(torch.FloatTensor):
    lit = getattr(torch.FloatTensor, attr)
    is_desc = inspect.ismethoddescriptor(lit)
    is_func = type(lit)==FunctionType
    is_mappingproxy = attr == '__dict__'
    try:
        is_service_func = 'TorchService' in lit.__qualname__ 
    except:
        is_service_func = False
    is_prop = isinstance(lit, property)
    is_bool = isinstance(lit, bool)
    is_base = lit in dir(object)
    is_none = lit is None
    if is_desc or (is_func and not is_service_func):
        print(attr, ' set')
    else:
        print(attr, ' skipped')

__add__  set
__and__  set
__array__  set
__array_wrap__  set
__bool__  set
__class__  skipped
__deepcopy__  set
__delattr__  set
__delitem__  set
__dict__  skipped
__dir__  set
__div__  set
__doc__  skipped
__eq__  set
__float__  set
__format__  set
__ge__  set
__getattribute__  set
__getitem__  set
__getstate__  set
__gt__  set
__hash__  set
__iadd__  set
__iand__  set
__idiv__  set
__ilshift__  set
__imul__  set
__init__  skipped
__init_subclass__  skipped
__int__  set
__invert__  set
__ior__  set
__ipow__  set
__irshift__  set
__isub__  set
__iter__  set
__itruediv__  set
__ixor__  set
__le__  set
__len__  set
__long__  set
__lshift__  set
__lt__  set
__matmul__  set
__mod__  set
__module__  skipped
__mul__  set
__ne__  set
__neg__  set
__new__  skipped
__nonzero__  set
__or__  set
__pow__  set
__radd__  set
__rdiv__  set
__reduce__  set
__reduce_ex__  set
__repr__  skipped
__rmul__  set
__rpow__  set
__rshift__  set
__rsub__  set
__rtruediv__  set
__setattr__  set
__setitem__  set


In [ ]:
imp.reload(torch)

In [17]:
torch.FloatTensor.is_sparse.__class__

bool

In [16]:
dir(torch.FloatTensor.is_sparse)

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes']

In [11]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [2]:
import torch

In [16]:
y = torch.FloatTensor([[2,2],[2,2]])
z = torch.FloatTensor([[1,1],[1,1]])

RecursionError: maximum recursion depth exceeded while calling a Python object

In [17]:
y.owner

'QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4'

In [18]:
z.owner

'QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4'

In [22]:
x = y.old_add(z)

AttributeError: 'torch.FloatTensor' object has no attribute 'old_add'

In [20]:
service_self.register_object(x, False)

owner registered



 3  3
 3  3
[torch.FloatTensor of size 2x2]

In [21]:
x.owner

'QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4'

In [20]:
y.is_pointer_to_remote = True
z.is_pointer_to_remote = True

In [21]:
x = y.add(z)

In [22]:
service_self.register_object(x, True)

owner registered



 3  3
 3  3
[torch.FloatTensor of size 2x2]

In [23]:
x


 3  3
 3  3
[torch.FloatTensor of size 2x2]

In [22]:
x.owner

'QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4'

In [21]:
client.register_object(x, False)

AttributeError: 'TorchClient' object has no attribute 'register_object'

In [12]:
args, kwargs = torch.mm(y, mat2 = z)

NameError: name 'y' is not defined

In [13]:
torch.add(y, other = z)

Placeholder print for sending command to worker A1
{'command': 'add', 'command_type': <class 'builtin_function_or_method'>, 'args': (
 2  2
 2  2
[torch.FloatTensor of size 2x2]
,), 'kwargs': {'other': 
 1  1
 1  1
[torch.FloatTensor of size 2x2]
}, 'arg_types': [<class 'torch.FloatTensor'>], 'kwarg_types': [<class 'torch.FloatTensor'>]}

Placeholder print for sending command to worker B1
{'command': 'add', 'command_type': <class 'builtin_function_or_method'>, 'args': (
 2  2
 2  2
[torch.FloatTensor of size 2x2]
,), 'kwargs': {'other': 
 1  1
 1  1
[torch.FloatTensor of size 2x2]
}, 'arg_types': [<class 'torch.FloatTensor'>], 'kwarg_types': [<class 'torch.FloatTensor'>]}

Placeholder print for sending command to worker B2
{'command': 'add', 'command_type': <class 'builtin_function_or_method'>, 'args': (
 2  2
 2  2
[torch.FloatTensor of size 2x2]
,), 'kwargs': {'other': 
 1  1
 1  1
[torch.FloatTensor of size 2x2]
}, 'arg_types': [<class 'torch.FloatTensor'>], 'kwarg_types': [<class '

((
   2  2
   2  2
  [torch.FloatTensor of size 2x2],), {'other': 
   1  1
   1  1
  [torch.FloatTensor of size 2x2]})

In [17]:
torch.add.__module__+'.'+torch.add.__name__

'torch._C.add'

In [18]:
import torch

In [21]:
print(torch.add.__name__ in dir(torch))

True


In [32]:
torch.FloatTensor.__module__

'torch'

In [22]:
dir(torch.FloatTensor.add)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__objclass__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__text_signature__']

In [2]:
def hook_var():
    def new___init__(self, *args, **kwargs):
        super(Variable, self).__init__(*args, **kwargs)
        self.id = random.randint(0, 1e10)
    Variable.__init__ = new___init__

In [3]:
def hook_add(worker_ids):
    @assign_workers(worker_ids)
    def new_add(self, other):
        frame = inspect.currentframe()
        command = compile_command('add', frame, self.id)
        return command
    
    @assign_workers(worker_ids)
    def new___add__(self, other):
        frame = inspect.currentframe()
        command = compile_command('__add__', frame, self.id)
        return command
    
    Variable.add = new_add
    Variable.__add__ = new___add__

In [33]:
for x in dir(torch):
    print(x, x in dir(torch.FloatTensor))

AutogradClosureFactory False
ByteStorage False
ByteTensor False
CharStorage False
CharTensor False
DoubleStorage False
DoubleTensor False
FatalError False
FloatStorage False
FloatTensor False
Generator False
Graph False
HalfStorage False
HalfStorageBase False
HalfTensor False
HalfTensorBase False
IntStorage False
IntTensor False
LongStorage False
LongTensor False
Node False
ShortStorage False
ShortTensor False
Size False
Storage False
Tensor False
TracingState False
Type False
Use False
_C False
_StorageBase False
_TensorBase False
__all__ False
__builtins__ False
__cached__ False
__doc__ True
__file__ False
__loader__ False
__name__ False
__package__ False
__path__ False
__spec__ False
__version__ False
_import_dotted_name False
_six False
_storage_classes False
_tensor_classes False
_tensor_str False
_thnn False
_utils False
abs True
acos True
add True
addbmm True
addcdiv True
addcmul True
addmm True
addmv True
addr True
arange False
asin True
atan True
atan2 True
autograd False
back

In [4]:
def assign_workers(worker_ids):
    def decorate(func):
        def send_to_workers(*args, **kwargs):
            command = func(*args, **kwargs)
            for worker in worker_ids:
                print("Placeholder print for sending command to worker {}".format(worker))
                send_command(command)
            receive_commands(worker_ids)  ## Probably needs to happen async
        return send_to_workers
    return decorate

In [6]:
def send_command(command):
    print(command)
    print('===========')
    print()

def receive_commands(worker_ids):
    print('Placeholder print for receiving commands from workers in the following list')
    print(worker_ids)
    print('For Compute and Tree mode, I think this is a single synchronous listen that terminates whenever the message is received')
    print('\t(I think, though I\'m still not quite familiar with how Tree works)')
    print('For Federated mode, this is an asynchronous listen, one thread to each worker.')
    print('\tTerminates when a stopping criterion is met.')
    print('Stopping criterion for synchronous SGD is when each worker has reported in.')
    print('Stopping criterion for asynchronous SGD is whenever the client decides to update the model')
    print('\t-- this will require some extra engineering to keep training going smoothlyin the meantime')

In [7]:
hook_var()
hook_add(['A1', 'A2', 'B1'])

In [8]:
x = Variable(torch.FloatTensor([[2,2],[2,2]]))

In [9]:
x.add(x)

Placeholder print for sending command to worker A1
{'tensor_id': 4983439554, 'command': 'add', 'args': ['self', 'other'], 'varargs': None, 'keywords': None, 'values': [Variable containing:
 2  2
 2  2
[torch.FloatTensor of size 2x2]
, Variable containing:
 2  2
 2  2
[torch.FloatTensor of size 2x2]
], 'types': [<class 'torch.autograd.variable.Variable'>, <class 'torch.autograd.variable.Variable'>]}

Placeholder print for sending command to worker A2
{'tensor_id': 4983439554, 'command': 'add', 'args': ['self', 'other'], 'varargs': None, 'keywords': None, 'values': [Variable containing:
 2  2
 2  2
[torch.FloatTensor of size 2x2]
, Variable containing:
 2  2
 2  2
[torch.FloatTensor of size 2x2]
], 'types': [<class 'torch.autograd.variable.Variable'>, <class 'torch.autograd.variable.Variable'>]}

Placeholder print for sending command to worker B1
{'tensor_id': 4983439554, 'command': 'add', 'args': ['self', 'other'], 'varargs': None, 'keywords': None, 'values': [Variable containing:
 2  2

In [10]:
x + x

Placeholder print for sending command to worker A1
{'tensor_id': 4983439554, 'command': '__add__', 'args': ['self', 'other'], 'varargs': None, 'keywords': None, 'values': [Variable containing:
 2  2
 2  2
[torch.FloatTensor of size 2x2]
, Variable containing:
 2  2
 2  2
[torch.FloatTensor of size 2x2]
], 'types': [<class 'torch.autograd.variable.Variable'>, <class 'torch.autograd.variable.Variable'>]}

Placeholder print for sending command to worker A2
{'tensor_id': 4983439554, 'command': '__add__', 'args': ['self', 'other'], 'varargs': None, 'keywords': None, 'values': [Variable containing:
 2  2
 2  2
[torch.FloatTensor of size 2x2]
, Variable containing:
 2  2
 2  2
[torch.FloatTensor of size 2x2]
], 'types': [<class 'torch.autograd.variable.Variable'>, <class 'torch.autograd.variable.Variable'>]}

Placeholder print for sending command to worker B1
{'tensor_id': 4983439554, 'command': '__add__', 'args': ['self', 'other'], 'varargs': None, 'keywords': None, 'values': [Variable conta

### Work in progress
The following stuff is ideally meant to automate the `hook_var` and `hook_add` functions and their ilk.
I've run into some trouble, mainly because there's no trivial way of reproducing some of the stuff above for functions that don't live completely in Python (e.g. functions that are considered 'built-in' because they originate somewhere in C).
Once I learn more about the `inspect` module, it may be possible.  To be continued...

In [4]:
def assign_name(func, name):
    func.name = name
    return func

def assign_sig(func, sig_func):
    func.sig = inspect.signature(sig_func)
    return func

def assign_argspec(func, spec_func):
    func.spec = inspect.getargspec(spec_func)

In [5]:
def generic(*args, **kwargs):
    frame = inspect.currentframe()
    command = compile_command(generic.name, frame)
    print(command)
    #send_command(command, worker)
    #receive_command(worker)

In [6]:
name = torch.addmm.__name__

In [7]:
generic = assign_sig(generic, torch.addmm)
generic = assign_name(generic, name)

ValueError: no signature found for builtin <built-in function addmm>

In [8]:
torch.autograd.Variable.add = generic

In [9]:
x = Variable(torch.FloatTensor([[2,2],[2,2]]))

In [10]:
x.add(other = x)

{'command': 'add', 'args': [], 'varargs': 'args', 'keywords': 'kwargs', 'values': [], 'types': []}


In [15]:
def dummy(x, y, z = 5):
     print(inspect.getargvalues(inspect.currentframe())[0])

In [16]:
dummy(1, 2, z = 5)

['x', 'y', 'z']


In [18]:
x = Variable(torch.FloatTensor([1,2,3,4]),requires_grad=True)

In [21]:
x.__class__.__name__

'Variable'

In [23]:
x.add.__name__

'add'

In [24]:
x.add(y)

Variable containing:
 3
 5
 7
 9
[torch.FloatTensor of size 4]

In [13]:
compile_command('add', )

TypeError: compile_command() takes 2 positional arguments but 3 were given

In [3]:
def send_command(command, *args, **kwargs):
    assert type(command) == str

In [ ]:
a, b = Var(x), Var(y)
a + b != Var(x + y)
Variable.__add__ != FloatTensor.__add__

In [2]:
def hook_var():
    
    Variable.lit___add__ = Variable.__add__ 
    def grid___add__(self,other):
        print("__add__")
        return self.lit___add__(other)
    Variable.__add__ = grid___add__
    
    Variable.lit_add = Variable.add
    def grid_add(self,other):
        print("add")
        return self.lit_add(other)
    Variable.add = grid_add

def hook_float():
    torch.FloatTensor.lit___add__ = torch.FloatTensor.__add__ 
    def grid___add__(self,other):
        print("__add__")
        return self.lit___add__(other)
    torch.FloatTensor.__add__ = grid___add__
    
    torch.FloatTensor.lit_add = torch.FloatTensor.add
    def grid_add(self,other):
        print("add")
        return self.lit_add(other)
    torch.FloatTensor.add = grid_add
    
def hook_torch():
    torch.lit_addmm = torch.addmm
    def grid_addmm(mat, mat1, mat2, beta=1, alpha=1):
        print("addmm")
        return torch.addmm(mat, mat1, mat2, beta = beta, alpha = alpha)

In [3]:
hook_float()

In [4]:
x = torch.FloatTensor([1,2,3,4])

In [5]:
x.add(x)

add



 2
 4
 6
 8
[torch.FloatTensor of size 4]

In [6]:
x + x

__add__
add



 2
 4
 6
 8
[torch.FloatTensor of size 4]

In [32]:
def graph():
    frames = inspect.stack()
    x = Variable(torch.FloatTensor([1,2,3,4]),requires_grad=True)
    y = Variable(torch.FloatTensor([2,3,4,5]),requires_grad=True)
    z = x.add(y)
    z.sum().backward()
    return x.grad, y.grad, frames

In [33]:
graph()

(Variable containing:
  1
  1
  1
  1
 [torch.FloatTensor of size 4], Variable containing:
  1
  1
  1
  1
 [torch.FloatTensor of size 4], [FrameInfo(frame=<frame object at 0x7f8b39eac600>, filename='<ipython-input-32-3c1f715c0d0c>', lineno=2, function='graph', code_context=['    frames = inspect.stack()\n'], index=0),
  FrameInfo(frame=<frame object at 0x7f8b39ed07c8>, filename='<ipython-input-33-76e6a91db885>', lineno=1, function='<module>', code_context=['graph()\n'], index=0),
  FrameInfo(frame=<frame object at 0x55a59652ca98>, filename='/home/jason/anaconda3/envs/openmined/lib/python3.6/site-packages/IPython/core/interactiveshell.py', lineno=2910, function='run_code', code_context=['                exec(code_obj, self.user_global_ns, self.user_ns)\n'], index=0),
  FrameInfo(frame=<frame object at 0x55a59652c828>, filename='/home/jason/anaconda3/envs/openmined/lib/python3.6/site-packages/IPython/core/interactiveshell.py', lineno=2856, function='run_ast_nodes', code_context=['      

In [13]:
y.grad

Variable containing:
 1
 1
 1
 1
[torch.FloatTensor of size 4]

In [4]:
# Notes -- very unstable.
# I think some of the things I'm replacing might have decorators, which could lead to some of the weird behavior
# We need to hook to these methods so that we can send out commands to workers
# Just need to be able to do so reliably
# May also need to disable the __str__ or print functionality on variables/tensors to avoid these RuntimeErrors

def hook_var():
    
    # None of the `__init__` or `new` stuff works yet
    curse(torch._C._VariableBase, "old__init__", torch._C._VariableBase.new)
    #curse(Variable, "old__init__", Variable.new)
    def _new___init__(self, *args):
        self.id = 1
        self.data_addr = 2
        print("init")
        #return self.old__init__(*args)
    #curse(torch._C._VariableBase, "__init__", _new___init__)
    curse(torch._C._VariableBase, "new", _new___init__)
    
    # `add` and `__add__` work sometimes, and even then not as expected
    curse(Variable, "old__add__", Variable.__add__)
    def _new___add__(self, other):
        print("__add__")
        #return self.old__add__(other)
    curse(Variable, "__add__", _new___add__)
   
    curse(torch._C._VariableBase, 'old_add', torch._C._VariableBase.add)
    def _new_add(self, other):
        print("add")
        #return self.old_add(other)
    curse(Variable, 'add', _new_add)
    
    # This works, usually
    torch.old_addmm = torch.addmm
    def _new_addmm(mat, mat1, mat2, beta=1, alpha=1):
        print("addmm")
        #return torch.old_addmm(beta, mat, alpha, mat1, mat2)
    torch.addmm = _new_addmm

In [5]:
def hook_torch():
    torch.tensor = grid_tensor
    torch.zeros = grid_zeros
    hook_var()

    
    #hook_long()

    # anything else we need to do to modify/override torch for the Client globally
    
    pass

In [6]:
hook_torch()

In [7]:
x = torch.tensor([2,2], requires_grad = True)

[2, 2]
None
None
True


In [8]:
x.id

AttributeError: 'Variable' object has no attribute 'id'

In [9]:
x.cuda()

RuntimeError: Overflow when unpacking long

In [10]:
x.add(x)

RuntimeError: Overflow when unpacking long

In [11]:
x + x

RuntimeError: Overflow when unpacking long

In [12]:
torch.addmm(x, x, x)

addmm


Below doesn't work -- can't subclass FloatTensor?

In [1]:
# torch.tensor(data, dtype=None, device=None, requires_grad=False)
# torch.zeros(shape, dtype=None, device=None, requires_grad=False)
# torch.ones(...)
# torch.empty(...)

In [1]:
import torch
from torch.autograd import Variable

In [15]:
class FakeVar(torch.autograd.variable):
    
    def __add__(self, other):
        return self.add(other)
    
    def add(self, other):
        print("add")


In [16]:
torch._C._VariableBase = FakeVar

In [20]:
torch.autograd.Variable = FakeVar

In [8]:
a = torch.tensor((2,2), dtype = torch.float, requires_grad = True)

(2, 2)
torch.float32
True


In [9]:
a.add(a)

__add__
__add__


RuntimeError: Overflow when unpacking long

In [23]:
type(a)

torch.autograd.variable.Variable

In [24]:
type(torch.tensor((2,2), requires_grad = True))

(2, 2)
None
True


torch.autograd.variable.Variable

In [25]:
a + a


 3.1621e-28  9.1370e-41
 3.1621e-28  9.1370e-41
[torch.FloatTensor of size (2,2)]